In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [2]:
url = 'https://liquipedia.net/valorant/VCT/2023/Champions/Statistics'

file = requests.get(url)

In [3]:
print(file.text)

<!DOCTYPE html>
<html class="client-nojs Send_pizza_to_FO-nTTaX All_glory_to_Liquipedia update-style" lang="en" dir="ltr" prefix="og: http://ogp.me/ns#">
<head>
<meta charset="UTF-8"/>
<title>VALORANT Champions 2023 - Statistics - Liquipedia VALORANT Wiki</title>
<script>document.documentElement.className="client-js Send_pizza_to_FO-nTTaX All_glory_to_Liquipedia update-style";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"41c6decf8c089bc18c81f3a3","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"VCT/2023/Champions/Statistics","wgTitle":"VCT/2023/Champions/Statistics","wgCurRevisionId":728422,"wgRevisionId":728422,"wgArticleId":42377,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserNam

In [4]:
# Parse the HTML content of the website
soup = BeautifulSoup(file.text, 'html.parser')

# Find the tables inside the data
soup.select('table')

[<table class="wikitable wikitable-striped sortable" style="margin-top:10px;text-align:center"><tbody><tr><th width="10">#</th><th width="200">Player</th><th width="50">Team</th><th width="75">Agents</th><th width="50">Maps</th><th width="50">K</th><th width="50">D</th><th width="50">A</th><th width="50">KD</th><th width="50">KDA</th><th width="50">ACS/Map</th><th width="50">K/Map</th><th width="50">D/Map</th><th width="50">A/Map</th></tr><tr><td>1</td><td style="text-align:left"><span class="flag"><a href="/valorant/Category:Chile" title="Chile"><img alt="Chile" decoding="async" height="24" src="/commons/images/thumb/8/8a/Cl_hd.png/36px-Cl_hd.png" srcset="/commons/images/thumb/8/8a/Cl_hd.png/54px-Cl_hd.png 1.5x, /commons/images/thumb/8/8a/Cl_hd.png/72px-Cl_hd.png 2x" width="36"/></a></span> <a href="/valorant/Keznit" title="Keznit">keznit</a></td><td><span class="team-template-team-icon" data-highlightingclass="KRÜ Esports"><span class="team-template-image-icon team-template-lightmode

# Player Statistics

In [5]:
# I then selected the first table which according to the website is their player statistics
table1 = soup.find('table', attrs={'class' :'wikitable wikitable-striped sortable'})

# I then extrated all the data in said row
table1_data = table1.tbody.find_all('tr')

In [6]:
# created a list to store the column headers
headings=[]

# Extracted the column headers from the first row
for td in table1_data[0].find_all('th'):

    #appended it into the list created earlier
    headings.append(td.text)
print(headings)

['#', 'Player', 'Team', 'Agents', 'Maps', 'K', 'D', 'A', 'KD', 'KDA', 'ACS/Map', 'K/Map', 'D/Map', 'A/Map']


In [7]:
#created a dictionary to store player demographics

demo={'Player':[],'Country':[],'Team':[],'Agents':[]}
final_stat = []

# looping through the rows starting from the second row
for i in range(1,len(table1_data)):
    temp = [] # temporary list for player demographics
    stat_temp = [] # temportary list for player stats

        
    # Loop through the cells in the current row then append the stats into the stat_temp file
    for td in table1_data[i].find_all('td'):
        stat_temp.append(td.text) 

                
        # Loop through the <a> element within the cell we are adding the title to the demo temp
        for t in td.find_all('a'):
            temp.append(t.get('title'))

    # using the dictionary we created earlier we extract and store players demographics to 
    # correspoinding column

    # although country doesn't have its own column the images have names corresponding to the country

    demo['Country'].append(temp[0])
    demo['Player'].append(temp[1])
    demo['Team'].append(temp[2])
    demo['Agents'].append(temp[4:])
    
    #i then append the players stats to the stat_temp file
    final_stat.append(stat_temp)

# creating data frames based on the players demographic and stats
info=pd.DataFrame(demo)
stat=pd.DataFrame(final_stat)

# since we didn't remove anything from the stat data frame we would need to drop the first 
# 4 columns as they are the demographics we seperated
stat.drop([0,1,2,3],axis=1,inplace=True)

# we then rename the columns using the headings list we created earlier on
stat.columns=headings[4:]



In [8]:
stat.head()

,Maps,K,D,A,KD,KDA,ACS/Map,K/Map,D/Map,A/Map
0,4,79,70,21,1.12,1.42,266,19.7,17.5,5.2
1,11,212,181,41,1.17,1.39,254,19.2,16.4,3.7
2,21,421,302,80,1.39,1.65,246,20,14.3,3.8
3,24,446,355,82,1.25,1.48,235,18.5,14.7,3.4
4,16,285,279,92,1.02,1.35,234,17.8,17.4,5.7


In [9]:
info.head()

,Player,Country,Team,Agents
0,Keznit,Chile,KRÜ Esports,"[Raze, Jett]"
1,Whz,China,Bilibili Gaming,"[Raze, Jett]"
2,Demon1,United States,Evil Geniuses,"[Jett, Astra, Chamber]"
3,Aspas,Brazil,LOUD,"[Raze, Jett]"
4,Jinggg,Singapore,Paper Rex,"[Raze, Phoenix]"


In [10]:
# now we can merge the datas together using the following comands
Player_Statistics = pd.merge(info, stat, left_index=True, right_index=True)

In [11]:
Player_Statistics

,Player,Country,Team,Agents,Maps,K,D,A,KD,KDA,ACS/Map,K/Map,D/Map,A/Map
0,Keznit,Chile,KRÜ Esports,"[Raze, Jett]",4,79,70,21,1.12,1.42,266,19.7,17.5,5.2
1,Whz,China,Bilibili Gaming,"[Raze, Jett]",11,212,181,41,1.17,1.39,254,19.2,16.4,3.7
2,Demon1,United States,Evil Geniuses,"[Jett, Astra, Chamber]",21,421,302,80,1.39,1.65,246,20,14.3,3.8
3,Aspas,Brazil,LOUD,"[Raze, Jett]",24,446,355,82,1.25,1.48,235,18.5,14.7,3.4
4,Jinggg,Singapore,Paper Rex,"[Raze, Phoenix]",16,285,279,92,1.02,1.35,234,17.8,17.4,5.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,Rin,China,Bilibili Gaming,"[Omen, Killjoy, Brimstone]",11,122,167,81,0.73,1.21,144,11,15.1,7.3
76,TZH,China,FunPlus Phoenix,"[Astra, Viper, Harbor]",4,42,59,23,0.71,1.1,137,10.5,14.7,5.7
77,Soulcas,United Kingdom,Team Liquid,"[Skye, Astra]",4,44,58,25,0.75,1.18,134,11,14.5,6.2
78,SugarZ3ro,Japan,ZETA DIVISION,"[Brimstone, Astra]",4,38,58,24,0.65,1.06,115,9.5,14.5,6


In [12]:
# i dont want to include the dataframe index(the row labels) so i set it to false
Player_Statistics.to_csv(path_or_buf='Data/Data - Player Statistics.csv',index=False)

# Maps Picked

In [13]:
# Parse the HTML content of the website
soup = BeautifulSoup(file.text, 'html.parser')

# Find the tables inside the data
soup.select('table')

[<table class="wikitable wikitable-striped sortable" style="margin-top:10px;text-align:center"><tbody><tr><th width="10">#</th><th width="200">Player</th><th width="50">Team</th><th width="75">Agents</th><th width="50">Maps</th><th width="50">K</th><th width="50">D</th><th width="50">A</th><th width="50">KD</th><th width="50">KDA</th><th width="50">ACS/Map</th><th width="50">K/Map</th><th width="50">D/Map</th><th width="50">A/Map</th></tr><tr><td>1</td><td style="text-align:left"><span class="flag"><a href="/valorant/Category:Chile" title="Chile"><img alt="Chile" decoding="async" height="24" src="/commons/images/thumb/8/8a/Cl_hd.png/36px-Cl_hd.png" srcset="/commons/images/thumb/8/8a/Cl_hd.png/54px-Cl_hd.png 1.5x, /commons/images/thumb/8/8a/Cl_hd.png/72px-Cl_hd.png 2x" width="36"/></a></span> <a href="/valorant/Keznit" title="Keznit">keznit</a></td><td><span class="team-template-team-icon" data-highlightingclass="KRÜ Esports"><span class="team-template-image-icon team-template-lightmode

In [14]:
# I then selected the first table which according to the website is their player statistics
table2 = soup.find('table', attrs={'class', 'table table-bordered'})

# I then extrated all the data in said row
table2_data = table2.tbody.find_all('tr')

In [15]:
final_data = []

# looping through the rows starting from the first row
for i in range(len(table2_data)):
    temp = [] # temporary list for player demographics
       
    # Loop through the cells in the current row then append the stats into the temp file
    for td in table2_data[i].find_all(['td','th']):
        temp.append(td.text) 
    
    # I then append the data to the temp file
    final_data.append(temp)

# creating data frames based on the map pick
map_pick=pd.DataFrame(final_data)

# Set the first row as column names
map_pick.columns = map_pick.iloc[0]

# Drop the first row 
map_pick.drop(0,inplace=True)

# Set the 'Map' column as the index
map_pick.set_index(keys='Map', inplace=True)


In [16]:
map_pick

,Total,Group D,Group B,Group C,Group A,Playoffs
Map,,,,,,
Ascent,16×,2×,4×,1×,1×,8×
Bind,19×,3×,4×,2×,3×,7×
Fracture,5×,1×,0×,1×,2×,1×
Haven,12×,1×,2×,3×,0×,6×
Lotus,16×,3×,1×,4×,2×,6×
Pearl,16×,1×,4×,1×,3×,7×
Split,22×,4×,0×,3×,4×,11×


In [17]:
# looking at the data we would need to remove the 'x' from the data

# therefore we iterate through all the columns inside the data 
# using apply and lambda we convert the string representations of values to integers by removing the last character
for i in map_pick.columns:
    map_pick[i]=map_pick[i].apply(lambda x:int(x[:-1]))

In [18]:
map_pick

,Total,Group D,Group B,Group C,Group A,Playoffs
Map,,,,,,
Ascent,16,2,4,1,1,8
Bind,19,3,4,2,3,7
Fracture,5,1,0,1,2,1
Haven,12,1,2,3,0,6
Lotus,16,3,1,4,2,6
Pearl,16,1,4,1,3,7
Split,22,4,0,3,4,11


In [19]:
map_pick.to_csv(path_or_buf='Data/Data - Map Picked.csv')

# Side Statistics

In [20]:
# Parse the HTML content of the website
soup = BeautifulSoup(file.text, 'html.parser')

# Find the tables inside the data
soup.select('table')

[<table class="wikitable wikitable-striped sortable" style="margin-top:10px;text-align:center"><tbody><tr><th width="10">#</th><th width="200">Player</th><th width="50">Team</th><th width="75">Agents</th><th width="50">Maps</th><th width="50">K</th><th width="50">D</th><th width="50">A</th><th width="50">KD</th><th width="50">KDA</th><th width="50">ACS/Map</th><th width="50">K/Map</th><th width="50">D/Map</th><th width="50">A/Map</th></tr><tr><td>1</td><td style="text-align:left"><span class="flag"><a href="/valorant/Category:Chile" title="Chile"><img alt="Chile" decoding="async" height="24" src="/commons/images/thumb/8/8a/Cl_hd.png/36px-Cl_hd.png" srcset="/commons/images/thumb/8/8a/Cl_hd.png/54px-Cl_hd.png 1.5x, /commons/images/thumb/8/8a/Cl_hd.png/72px-Cl_hd.png 2x" width="36"/></a></span> <a href="/valorant/Keznit" title="Keznit">keznit</a></td><td><span class="team-template-team-icon" data-highlightingclass="KRÜ Esports"><span class="team-template-image-icon team-template-lightmode

In [21]:
# I then selected the first table which according to the website is their player statistics
table3 = soup.find('table', attrs={'class', 'wikitable wikitable-bordered wikitable-striped'})

# I then extrated all the data in said row
table3_data = table3.tbody.find_all('tr')

In [22]:
final_data = []

# looping through the rows starting from the first row
for i in range(len(table3_data)):
    temp = [] # temporary list for player demographics
       
    # Loop through the cells in the current row then append the stats into the temp file
    for td in table3_data[i].find_all(['td','th']):
        temp.append(td.text) 
    
    # I then append the data to the temp file
    final_data.append(temp)

# creating data frames based on the map pick
side_stat=pd.DataFrame(final_data)

# Set the first row as column names
side_stat.columns = side_stat.iloc[0]

# Drop the first row 
side_stat.drop(0,inplace=True)

# Set the 'Map' column as the index
side_stat.set_index(keys='Map', inplace=True)


In [23]:
side_stat

,Atk Wins,Def Wins
Map,,
Ascent,145 (51.2%),138 (48.8%)
Bind,157 (46.2%),183 (53.8%)
Fracture,51 (49.0%),53 (51.0%)
Haven,107 (49.5%),109 (50.5%)
Lotus,151 (55.1%),123 (44.9%)
Pearl,149 (49.7%),151 (50.3%)
Split,178 (56.3%),138 (43.7%)


In [24]:
# removing the percentage and brackets from thle table

# selecting the columns we would want to split the string values into 
# seperate ones then only select the first element in said list 
# the code would have the data as ('145','(46.3%)') then we only pick the first in this list
side_stat['Atk Wins']=side_stat['Atk Wins'].apply(lambda x:int(x.split()[0]))
side_stat['Def Wins']=side_stat['Def Wins'].apply(lambda x:int(x.split()[0]))

In [25]:
side_stat

,Atk Wins,Def Wins
Map,,
Ascent,145,138
Bind,157,183
Fracture,51,53
Haven,107,109
Lotus,151,123
Pearl,149,151
Split,178,138


In [26]:
side_stat.to_csv(path_or_buf='Data/Data - Side Statistics.csv')

# Maps Banned

In [27]:
# Parse the HTML content of the website
soup = BeautifulSoup(file.text, 'html.parser')

# Find the tables inside the data
soup.select('table')

[<table class="wikitable wikitable-striped sortable" style="margin-top:10px;text-align:center"><tbody><tr><th width="10">#</th><th width="200">Player</th><th width="50">Team</th><th width="75">Agents</th><th width="50">Maps</th><th width="50">K</th><th width="50">D</th><th width="50">A</th><th width="50">KD</th><th width="50">KDA</th><th width="50">ACS/Map</th><th width="50">K/Map</th><th width="50">D/Map</th><th width="50">A/Map</th></tr><tr><td>1</td><td style="text-align:left"><span class="flag"><a href="/valorant/Category:Chile" title="Chile"><img alt="Chile" decoding="async" height="24" src="/commons/images/thumb/8/8a/Cl_hd.png/36px-Cl_hd.png" srcset="/commons/images/thumb/8/8a/Cl_hd.png/54px-Cl_hd.png 1.5x, /commons/images/thumb/8/8a/Cl_hd.png/72px-Cl_hd.png 2x" width="36"/></a></span> <a href="/valorant/Keznit" title="Keznit">keznit</a></td><td><span class="team-template-team-icon" data-highlightingclass="KRÜ Esports"><span class="team-template-image-icon team-template-lightmode

In [28]:
# I then selected the first table which according to the website is their player statistics
table4 = soup.find('table', attrs={'class', 'table table-bordered'})

#since there are duplicate tables with the same class we use the command find_next
table4 = table4.find_next('table').find_next('table')

# I then extrated all the data in said row
table4_data = table4.tbody.find_all('tr')

In [29]:
final_data = []

# looping through the rows starting from the first row
for i in range(len(table4_data)):
    temp = [] # temporary list for player demographics
       
    # Loop through the cells in the current row then append the stats into the temp file
    for td in table4_data[i].find_all(['td','th']):
        temp.append(td.text) 
    
    # I then append the data to the temp file
    final_data.append(temp)

# creating data frames based on the map pick
map_ban=pd.DataFrame(final_data)

# Set the first row as column names
map_ban.columns = map_ban.iloc[0]

# Drop the first row 
map_ban.drop(0,inplace=True)

# Set the 'Map' column as the index
map_ban.set_index(keys='Map', inplace=True)


In [30]:
map_ban

,Total,Group D,Group B,Group C,Group A,Playoffs
Map,,,,,,
-,2×,0×,0×,0×,0×,2×
Ascent,18×,3×,1×,4×,4×,6×
Bind,15×,2×,1×,3×,2×,7×
Fracture,29×,4×,5×,4×,3×,13×
Haven,22×,4×,3×,2×,5×,8×
Lotus,18×,2×,4×,1×,3×,8×
Pearl,18×,4×,1×,4×,2×,7×
Split,12×,1×,5×,2×,1×,3×


In [31]:
# looking at the data we would need to remove the 'x' from the data

# therefore we iterate through all the columns inside the data 
# using apply and lambda we convert the string representations of values to integers by removing the last character
for i in map_ban.columns:
    map_ban[i]=map_ban[i].apply(lambda x:int(x[:-1]))

In [32]:
map_ban

,Total,Group D,Group B,Group C,Group A,Playoffs
Map,,,,,,
-,2,0,0,0,0,2
Ascent,18,3,1,4,4,6
Bind,15,2,1,3,2,7
Fracture,29,4,5,4,3,13
Haven,22,4,3,2,5,8
Lotus,18,2,4,1,3,8
Pearl,18,4,1,4,2,7
Split,12,1,5,2,1,3


In [33]:
map_ban.to_csv(path_or_buf='Data/Data - Map Banned.csv')